In [2]:
!pip uninstall -y psycopg2-binary
!pip install psycopg2-binary==2.8.6



Found existing installation: psycopg2-binary 2.9.10
Uninstalling psycopg2-binary-2.9.10:
  Successfully uninstalled psycopg2-binary-2.9.10
     |████████████████████████████████| 1.5 MB 5.5 MB/s eta 0:00:01
You should consider upgrading via the '/Users/syoung/opt/anaconda3/bin/python3 -m pip install --upgrade pip' command.


In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2
import getpass
from IPython.display import display, HTML
# %matplotlib inline
plt.style.use('ggplot') 


In [5]:
# information used to create a database connection
sqluser = 'postgres'
dbname = 'mimic'
schema_name = 'mimiciii'

# Connect to postgres with a copy of the MIMIC-III database
con = psycopg2.connect(dbname=dbname, user=sqluser)

# the below statement is prepended to queries to ensure they select from the right schema
query_schema = 'set search_path to ' + schema_name + ';'

OperationalError: FATAL:  role "postgres" does not exist


In [ ]:
demographics_query = """
    SELECT p.subject_id, p.gender, p.dob,
           a.hadm_id, a.admittime, a.dischtime, a.deathtime, a.admission_type,
           i.icustay_id, i.intime, i.outtime, i.los
    FROM patients p
    JOIN admissions a
      ON p.subject_id = a.subject_id
    JOIN icustays i
      ON a.subject_id = i.subject_id
     AND a.hadm_id = i.hadm_id;
    """

demographic_data = pd.read_sql_query(demographics_query, con)
demographic_data.head()

In [ ]:
lab_query = """
    SELECT subject_id, hadm_id, itemid, valuenum, charttime
    FROM labevents
    WHERE valuenum IS NOT NULL;
    """

lab_data = pd.read_sql_query(lab_query, con)
lab_data.head()

In [ ]:
history_query = """
    SELECT subject_id, hadm_id, icd9_code
    FROM diagnoses_icd;
    """

history_data = pd.read_sql_query(history_query, con)
history_data.head()

In [6]:
pd.merge(demographic_data, lab_data, on=['subject_id', 'hadm_id'], how='left')

NameError: name 'demographics' is not defined